In [6]:
import pandas as pd
import numpy as np
import re

In [7]:
dir = "/Users/mckennaquam/Desktop/DS 4420/ML4420_project/"


df = pd.read_csv(dir + "data/athlete_events.csv")
df = df[df["Sport"] == "Athletics"]
df = df.dropna(subset=['Age', 'Height', 'Weight', 'Sex', 'Event'])
df = df[["Age", "Height", "Weight", "Sex", "Event"]]

In [8]:
# field events, shot put jav, anything throwing
# spirts under 400
# middle distance 800 - mile
# long 5k +
# jumping pole value, high jump, long jump


def sort_athletics(e):
    e = e.lower()

    if (any(event in e for event in ["jump", "vault"])):
        return "Jump"
    elif (any(event in e for event in ["discus", "javelin", "shot put", "throw"])):
        return "Field"
    elif (any(event in e for event in ["kilometre", "cross-country", "marathon"])):
        return "Long"
    
    elif ("mile" in e):
        match = re.search(r'(\d[\d,]*)\s+mile', e)
        if match:
            length = int(match.group(1).replace(',', ''))
        else:
            print(f"ERROR: {e}") 

        if (length <= 1):
            return "Mid"
        else:
            return "Long"

    elif ("metre" in e):

        match = re.search(r'(\d[\d,]*)\s+metre', e)
        if match:
            length = int(match.group(1).replace(',', ''))
        else:
            print(f"ERROR: {e}") 

        if length <= 400:
            return "Short"
        elif (length > 400 and  length <= 1609.34):
            return "Mid"
        else:
            return "Long"
    else:
        # disgarding Decathalon, Pentathelon, Heptathlon, All Around
        return "Not Classed"
    

df["event_catagory"] = df["Event"].apply(sort_athletics)

In [9]:
df.groupby("event_catagory").size()

event_catagory
Field           3995
Jump            4142
Long            7283
Mid             2889
Not Classed     1080
Short          12985
dtype: int64

In [10]:
# rest of the data cleaning
df = df[df["event_catagory"] != "Not Classed"]
df = df.drop('Event', axis=1)

df["Sex_F"] = np.where(df['Sex'] == "F", 1, 0)
df["Sex_M"] = np.where(df['Sex'] == "M", 1, 0)
df = df.drop('Sex', axis=1)

def scale(x):
    return (x - min(x)) / (max(x) - min(x))

df[["Age", "Height", "Weight"]] = df[["Age", "Height", "Weight"]].apply(scale)


df.to_csv(dir + "data/athletics_cleaned.csv")